In [776]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

schools_path = os.path.join("Resources" , "schools_complete.csv")
students_path = os.path.join ("Resources", "students_complete.csv")

schools_data = pd.read_csv(schools_path)
students_data = pd.read_csv(students_path)

data_complete = pd.merge(students_data, schools_data, 
                how="left", 
                on=["school_name", "school_name"])\
                .rename(columns=lambda s:s.lower().replace(" ", "_"))  
data_complete.head(30)

,student_id,student_name,gender,grade,school_name,reading_score,math_score,school_id,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635


In [777]:
total_schools = len(data_complete["school_name"].unique())
total_students = len(data_complete["student_id"].unique())
district_budget = data_complete.drop_duplicates('school_name')['budget'].sum()

math_average = round(data_complete["math_score"].mean())
reading_average = round(data_complete["reading_score"].mean())

# Total number of passed students in each class
students_passing_math = data_complete[data_complete["math_score"] >= 70].count()["student_id"]
students_passing_reading = data_complete[data_complete["reading_score"] >= 70].count()["student_id"]

percentage_students_passed_math = round((students_passing_math/total_students)*100,2)
percentage_students_passed_reading = round((students_passing_reading/total_students)*100,2)

# Porcentage of passed students in the district
district_overall_passing_rate = (percentage_students_passed_math + percentage_students_passed_reading)/2

d = {"Summary":["Total Schools", "Total Students", "District Budget", "District Passing Rate"],
    " ":[total_schools, total_students, district_budget, district_overall_passing_rate]}

d2 = {"Summary": ["Average Grade", "Passing Students"],
      "Math": [math_average, percentage_students_passed_math],
      "Reading": [reading_average, percentage_students_passed_reading]}
district_summary = pd.DataFrame(d)
education_summary = pd.DataFrame(d2)

In [778]:
school_type = schools_data.set_index(["school_name"])["type"]  
total_students_per_school = data_complete.groupby(["school_name"]).mean()["size"]
budget_per_school = data_complete.groupby(["school_name"]).mean()["budget"]
budget_per_student = budget_per_school/total_students_per_school

math_average_score_per_school = round(data_complete.groupby(["school_name"]).mean()["math_score"],2)
reading_average_score_per_school = round(data_complete.groupby(["school_name"]).mean()["reading_score"],2)

students_passed_math = data_complete[(data_complete["math_score"]>=70)]
students_passed_math_per_school = students_passed_math.groupby(["school_name"]).count()["student_id"]

students_passed_reading = data_complete[(data_complete["reading_score"]>=70)]
students_passed_reading_per_school = students_passed_reading.groupby(["school_name"]).count()["student_id"]

percentage_passed_math_per_school = round((students_passed_math_per_school/total_students_per_school)*100,2)
percentage_passed_reading_per_school = round((students_passed_reading_per_school/total_students_per_school)*100,2)

overall_passed_rate_per_school = (percentage_passed_math_per_school + percentage_passed_reading_per_school)/2

d3 = {
      "School Type":school_type, 
      "Number of Students": total_students_per_school, 
      "Budget per School":budget_per_school, 
      "Budget per Student":budget_per_student,
      "Average Math Score": math_average_score_per_school,
      "Average Reading Score":reading_average_score_per_school,
      "% Passing Math": percentage_passed_math_per_school,
      "% Passing Reading": percentage_passed_reading_per_school,
      "% Overall Passing Rate":overall_passed_rate_per_school
      }

school_summary = pd.DataFrame(d3)


In [779]:
school_type

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

## District Summary

In [780]:
district_summary

,Summary,
0,Total Schools,1.500000e+01
1,Total Students,3.917000e+04
2,District Budget,2.464943e+07
3,District Passing Rate,8.039500e+01


In [781]:
education_summary

,Summary,Math,Reading
0,Average Grade,79.00,82.00
1,Passing Students,74.98,85.81


## School Summary

In [782]:
school_summary

,School Type,Number of Students,Budget per School,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976.0,3124928.0,628.0,77.05,81.03,66.68,81.93,74.305
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.06,83.98,94.13,97.04,95.585
Figueroa High School,District,2949.0,1884411.0,639.0,76.71,81.16,65.99,80.74,73.365
Ford High School,District,2739.0,1763916.0,644.0,77.10,80.75,68.31,79.30,73.805
Griffin High School,Charter,1468.0,917500.0,625.0,83.35,83.82,93.39,97.14,95.265
Hernandez High School,District,4635.0,3022020.0,652.0,77.29,80.93,66.75,80.86,73.805
Holden High School,Charter,427.0,248087.0,581.0,83.80,83.81,92.51,96.25,94.380
Huang High School,District,2917.0,1910635.0,655.0,76.63,81.18,65.68,81.32,73.500
Johnson High School,District,4761.0,3094650.0,650.0,77.07,80.97,66.06,81.22,73.640
Pena High School,Charter,962.0,585858.0,609.0,83.84,84.04,94.59,95.95,95.270


## Top 5 Performing Schools (By Passing Rate)

In [783]:
top_schools = school_summary.sort_values(by="% Overall Passing Rate", ascending=False)
top_schools.head(5)

#budget_per_school.map("${:,.2f}".format)

,School Type,Number of Students,Budget per School,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.06,83.98,94.13,97.04,95.585
Thomas High School,Charter,1635.0,1043130.0,638.0,83.42,83.85,93.27,97.31,95.290
Pena High School,Charter,962.0,585858.0,609.0,83.84,84.04,94.59,95.95,95.270
Griffin High School,Charter,1468.0,917500.0,625.0,83.35,83.82,93.39,97.14,95.265
Wilson High School,Charter,2283.0,1319574.0,578.0,83.27,83.99,93.87,96.54,95.205


## Worst 5 Performing Schools (By Passing Rate)

In [784]:
bottom_schools = school_summary.sort_values(by="% Overall Passing Rate", ascending=True)
bottom_schools.head(5)

,School Type,Number of Students,Budget per School,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999.0,2547363.0,637.0,76.84,80.74,66.37,80.22,73.295
Figueroa High School,District,2949.0,1884411.0,639.0,76.71,81.16,65.99,80.74,73.365
Huang High School,District,2917.0,1910635.0,655.0,76.63,81.18,65.68,81.32,73.500
Johnson High School,District,4761.0,3094650.0,650.0,77.07,80.97,66.06,81.22,73.640
Ford High School,District,2739.0,1763916.0,644.0,77.10,80.75,68.31,79.30,73.805


## Math Scores by Grade per School

In [879]:
# filter per grade
grade_9 = data_complete[data_complete["grade"] == "9th"]
grade_10 = data_complete[data_complete["grade"] == "10th"]
grade_11 = data_complete[data_complete["grade"] == "11th"]
grade_12 = data_complete[data_complete["grade"] == "12th"]

# Average math scores per grade per school

grade_9_math_per_school = round(grade_9.groupby(["school_name"]).mean()["math_score"], 3)
grade_10_math_per_school = round(grade_10.groupby(["school_name"]).mean()["math_score"], 3)
grade_11_math_per_school = round(grade_11.groupby(["school_name"]).mean()["math_score"], 3)
grade_12_math_per_school = round(grade_12.groupby(["school_name"]).mean()["math_score"], 3)

# Combine the grades

avg_math_grade_per_school = pd.DataFrame ({
        "9th": grade_9_math_per_school,
        "10th":grade_10_math_per_school,
        "11th":grade_11_math_per_school,            
        "12th":grade_12_math_per_school
           })
                                            
avg_math_grade_per_school

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863


In [872]:
  
  
    
    
    {"Summary": ["Average Grade", "Passing Students"],
      "Math": [math_average, percentage_students_passed_math],
      "Reading": [reading_average, percentage_students_passed_reading]}
    

#avg_math_scores_grade = round(data_complete.groupby(["grade"]).mean()["math_score"],2)
#avg_math_scores_grade

{'Summary': ['Average Grade', 'Passing Students'],
 'Math': [79.0, 74.98],
 'Reading': [82.0, 85.81]}

## Reading Score by Grade 

In [880]:
# Average math scores per grade per school

grade_9_reading_per_school = round(grade_9.groupby(["school_name"]).mean()["reading_score"], 3)
grade_10_reading_per_school = round(grade_10.groupby(["school_name"]).mean()["reading_score"], 3)
grade_11_reading_per_school = round(grade_11.groupby(["school_name"]).mean()["reading_score"], 3)
grade_12_reading_per_school = round(grade_12.groupby(["school_name"]).mean()["reading_score"], 3)

# Combine the grades

avg_reading_grade_per_school = pd.DataFrame ({
        "9th": grade_9_reading_per_school,
        "10th":grade_10_reading_per_school,
        "11th":grade_11_reading_per_school,            
        "12th":grade_12_reading_per_school
           })
                                            
avg_reading_grade_per_school

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [759]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [760]:
data_complete[data_complete["math_score"]>=70]

,student_id,student_name,gender,grade,school_name,reading_score,math_score,school_id,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## Scores by School Size

* Perform the same operations as above, based on school size.

In [761]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.